# 脳波データを活用した運動意図の判定

これは「脳波データを活用した運動意図の判定」の分析・モデリングチュートリアルである. 必要に応じて分析・モデリングをする際の参考にされたい.

## 前準備

### データの準備

配布されている`train_0.zip~train_5.zip`, `train_master.csv`, `test_0.zip~test_2.zip`をダウンロードし, このノートブックと同じディレクトリに配置して, zipファイルは解凍する. 解凍後以下のようなディレクトリができていることを確認.

```
.
├── train              # 学習用データ
│   ├── train_000.csv
│   ├── ...
│   └── train_2247.csv
├── test               # 評価用データ
│   ├── test_000.csv
│   ├── ...
│   └── test_757.csv
├── train_master.csv   # 学習用アノテーションデータ
└── tutorial.ipynb     # このノートブックファイル
```

各データの定義については配布されている`README.pdf`を参照すること.

#### Google Colaboratoryを使う場合

自身のドライブなどにこのjupyternotebookファイルをアップロードして立ち上げることでGoogle Colaboratoryを起動し, `/content`以下に`train_0.zip~train_5.zip`, `train_master.csv`, `test_0.zip~test_2.zip`をアップロードする. その後下記コマンドを実行することで, zipファイルは解凍される. またはGoogle Driveをマウントしてもよい.

In [ ]:
# 現在のディレクトリを表示
! pwd # デフォルトなら`/content`と表示される
! ls  # train_*.zip, test_*.zipが表示されることを確認

In [ ]:
# Google Driveをマウントしているならターミナルからドライブのディレクトリへ移動して`unzip`を実行する.
# または解凍したものをフォルダごとドライブにアップロードしておく(その場合は下記コマンドを実行する必要はない.).
! unzip train_*.zip
! unzip test_*.zip

### ライブラリのインストール

これから行う分析やモデリングに必要なライブラリをインストールする. 主に必要なライブラリは以下の通り.

- pandas
- numpy
- matplotlib
- mne
- braindecode

インストールされていないなら下記コマンドでインストールすること.

In [ ]:
! pip install braindecode

`braindecode`をインストールすることで他のライブラリは自動的にインストールされる(依存ライブラリのため)

## ライブラリのインポートと設定

今後の作業で必要になる「道具」（＝Pythonのライブラリ）を用意する. また, データやファイルを保存しているフォルダの場所（ルートディレクトリ）を決めておく. これを最初にやっておくと, 今後データを読み込んだり保存したりするときに, 毎回場所を指定しなくてすむので便利である.

下記セルのコメントアウトにてインポートしているライブラリの役割を簡単に確認されたい.

In [ ]:
# データ分析や機械学習に使う道具（ライブラリ）を準備します
import os                          # ファイルやフォルダを操作する
import pandas as pd                # 表形式のデータを扱う
import numpy as np                 # 数値計算や配列の操作をする
import matplotlib.pyplot as plt    # グラフや図を描く
import mne                         # 脳波データを分析する

# braindecodeは脳波などの時系列信号用・ディープラーニングモデルを使うためのライブラリです.
# EEGClassifierはPyTorchモデルを簡単に分類問題に使えるようにまとめたクラスです.
from braindecode import EEGClassifier

# skorchはPyTorchをscikit-learnのような使い方で便利にするためのラッパーです.
# ValidSplitはデータを自動で訓練用と検証用に分割して管理するためのクラスです.
from skorch.dataset import ValidSplit


# データの読み込み場所, 結果の保存場所を決めておきます
DATA_DIR = '.'    # データファイルがあるフォルダ（ここではカレントフォルダを指定. Google Colaboratoryを使っていてGoogle Driveをマウントしてデータを展開しているなら`/content/drive/MyDrive`などと設定）
OUTPUT_DIR = '.'  # 結果などを保存するフォルダ（ここではカレントフォルダを指定. Google Colaboratoryを使っていてGoogle Driveをマウントしてデータを展開しているなら`/content/drive/MyDrive`などと設定）

## データの読み込み

これから分析に使うためのデータファイル（主にCSVファイル）をプログラムに読み込む. データを読み込むことで, Pythonからデータの中身を調べたり, いろいろな分析をできるようになる. 

### 学習用

学習用として与えられているのは`./train`以下にあるEEGの系列データと対応するラベル情報`train_master.csv`である. 以下の処理を行う.

- 分類で使うラベル名と番号（id）を対応させて定義する.
- 学習用データ（EEGデータ）ファイルを1つずつ読み込み, `np.array`形式のデータセット（train_X, train_y）を作る.
- 読み込んだデータの中身や, サンプル数, チャンネル数, 系列長, データ型などの特徴を確認する.

これらは機械学習モデリングを行うための準備である.

In [ ]:
# データファイルの場所（ファイル名：train_master.csv）を指定します
data_path = os.path.join(DATA_DIR, 'train_master.csv')

# pandasのread_csvを使って, ファイルを読み込みます
train_df = pd.read_csv(data_path)

# ちゃんと読み込めたか, 最初の5行を表示して確認します
print(train_df.head(), '\n')

# ラベルごとに名前を割り当ててdictで定義します. 安静時(nonactive)は0, 運動想起時(active)は1.
id2label = {0: 'nonactive', 1: 'active'}

# 辞書データの内容をprintで確認
print('labels:', id2label, '\n')

# 学習用の音声データを1ファイルずつ読み込み, リストとしてデータセットを作成します
train_X = []  # 入力データを入れるリスト
train_y = []  # 対応するラベルidを入れるリスト

for i, d in train_df.iterrows():  # 学習用データフレーム（train_df）から1行ずつ取り出す
    fpath = os.path.join(DATA_DIR, 'train', d['id']+'.csv')  # ファイルのパスを作成
    x = pd.read_csv(fpath, header=None)  # データ(csv)を読み込む
    numna = x.isna().sum().sum() # 欠損の確認
    if numna:
        print(i, 'includes na', numna)
    train_X.append(x.values.T)  # データを追加
    train_y.append(d['label'])  # ラベルを追加

# リストをnumpy配列に変換（機械学習で使いやすい形）
train_X = np.array(train_X)
train_y = np.array(train_y)

# データの特徴や配列サイズなどを確認
n_samples, num_channels, seq_len = train_X.shape  # サンプル数とチャンネル数と系列長
dtype = train_X.dtype  # データ型

print('min:', train_X.min(), 'max:', train_X.max())  # 最小・最大値
print('n_samples:', n_samples)  # サンプル数
print('num_channels:', num_channels) # チャンネル数
print('seq_len:', seq_len) # 系列データの長さ
print('dtype:', dtype)  # データ型

### 評価用

評価用として与えられているのは`./test`以下にあるEEGの系列データのみである. ラベル情報は与えられず, このラベルを当てることが今回の課題の主目的である. ここでは学習用データと同様に以下の処理を行う.

- テスト用（評価用）のEEGデータファイルをすべて読み込み, np.array形式の配列（test_X）を作る.
- 各ファイルのid（ファイル名から取得）もリスト（test_id）として記録する.
- 読み込んだデータのサイズやidリストを表示し, 正しく取り込めたか確認する.

これらは学習した機械学習モデルによって評価用データに対して推論を行って, 結果を作成するための準備である.

In [ ]:
test_id = []  # ファイルごとのidを入れるリスト
test_X = []   # 脳波データ本体を入れるリスト

# testフォルダにある全てのファイルをid順に処理します
for p in sorted(os.listdir(os.path.join(DATA_DIR, 'test'))):  # ファイル名で順に並べる
    fpath = os.path.join(DATA_DIR, 'test', p)  # ファイルのパスを作成
    x = pd.read_csv(fpath, header=None)  # ファイルを読み込む
    test_X.append(x.values.T)             # データ本体をリストに追加（転置して(チャンネル数×系列長)に）
    test_id.append(p.split('.')[0])       # ファイル名からid部分だけ抜き出してリストに追加

# リストをnumpy配列に変換します
test_X = np.array(test_X)

# 読み込んだデータの形状・idリストを表示して確認します
print(test_X.shape)  # データの形（サンプル数 × チャンネル数 ×長さ）
print(test_id)       # ファイルのidリスト

## 可視化・前処理

読み込んだデータを可視化したり, 適切な前処理を行う.

### 波形の可視化

学習データのうち1つのサンプルについて, 波形（時間と振幅のグラフ）を表示する. 波形を可視化することで, データの特徴やノイズ, 異常がないかを直感的に確認できる.

In [ ]:
sr = 100          # サンプリングレート（1秒あたりのデータ数）
length = 500      # 可視化するデータ数（5秒分）
idx = 0           # 可視化したいサンプルのインデックス（0番目を指定）

# チャンネル名と型
ch_names = [f'EEG {i}' for i in range(num_channels)]
ch_types = ['eeg'] * num_channels

# Infoの作成
info = mne.create_info(ch_names=ch_names, sfreq=sr, ch_types=ch_types) # type: ignore

# RawArrayの作成
raw = mne.io.RawArray(train_X[idx]*1e-6, info) # [μV]から[V]に変換

# 生データの可視化
raw.plot(n_channels=num_channels, scalings='auto', title='EEG from NumPy array')

plt.show() # グラフ表示

`idx`を変えてほかのサンプルの波形も確認してみるとよい. 他にもパワースペクトル密度の可視化などさらなる詳細な分析も可能. 詳しくは[mneのチュートリアル](https://mne.tools/stable/auto_tutorials/index.html)を参照すること.

### ラベルの分布可視化

学習データに含まれるラベル（クラス）がどれくらいの数ずつあるかを棒グラフで可視化する. データの偏り（クラスバランス）があるかどうかを事前に確認するためのステップである.

In [ ]:
# 各ラベルごとの件数を集計します
vals, cnts = np.unique(train_y, return_counts=True)  # ラベルIDごとのユニーク値とそれぞれの出現回数を取得

# ラベルIDをラベル名（文字列）に変換します
names = [id2label.get(v, f"ID{v}") for v in vals]

# 棒グラフを作成して、ラベルごとの個数を可視化
plt.figure(figsize=(8,4))  # グラフのサイズを指定
plt.bar(names, cnts, width=0.8, align='center')  # 横軸＝ラベル名、縦軸＝件数
plt.xlabel('label name')    # x軸ラベル
plt.ylabel('count')         # y軸ラベル
plt.show()                  # グラフ表示

### 前処理

各サンプルごとに, 波形データの平均が0, 標準偏差が1になるように標準化（正規化）する. これにより, 値のスケールの違いが学習に影響しにくくなり, 分析や機械学習でより良い結果が得やすくなる.

In [ ]:
# --- 学習用データの前処理 ---
# 前処理済みのデータを保存するための配列を用意（サイズは元データと同じ）
train_X_preprocessed = np.zeros((n_samples, num_channels, seq_len))


# 各サンプルごとに標準化（平均を0、標準偏差を1に）を行う
for i in range(len(train_X)):
    # 平均を引いて値を0中心にし、標準偏差で割って値のばらつきを揃える
    train_x = np.where(np.isnan(train_X[i]), np.nanmean(train_X[i]), train_X[i]) # 欠損値は平均値で補完する
    mean = train_x.mean(axis=1, keepdims=True)
    std = train_x.std(axis=1, keepdims=True)
    std[np.where(std==0)] = 1
    train_X_preprocessed[i] = (train_x - mean) / std
print(train_X_preprocessed.shape)

# --- 評価用データの前処理 ---
# 前処理済みのデータを保存するための配列を用意（サイズは元データと同じ）
test_n_samples, num_channels, seq_len = test_X.shape # type: ignore
test_X_preprocessed = np.zeros((test_n_samples, num_channels, seq_len))

# 各サンプルごとに標準化（平均を0、標準偏差を1に）を行う
for i in range(len(test_X)):
    # 平均を引いて値を0中心にし、標準偏差で割って値のばらつきを揃える
    test_x = np.where(np.isnan(test_X[i]), np.nanmean(test_X[i]), test_X[i]) # 欠損値は平均値で補完する
    mean = test_x.mean(axis=1, keepdims=True)
    std = test_x.std(axis=1, keepdims=True)
    std[np.where(std==0)] = 1
    test_X_preprocessed[i] = (test_x - mean) / std
print(test_X_preprocessed.shape)

## モデル構築・学習

学習データを使って, 運動意図を予測できるように深層学習モデルを学習させる. ここではShallowFBCSPNetというモデルを使い, データを訓練・検証に分けて自動で評価も行う.

In [ ]:
# 深層学習モデル(ShallowFBCSPNet)を作成
# - batch_size: 学習時に一度に使うサンプル数
# - max_epochs: 学習の繰り返し回数
# - train_split: 学習データの25%を検証用として自動で分ける
net = EEGClassifier(
    "ShallowFBCSPNet",
    batch_size=32,
    max_epochs=20,
    train_split=ValidSplit(0.25) # 検証データ割合 # type: ignore
)

# 学習データ（train_X_preprocessed, train_y）を使ってモデルを訓練
net.fit(train_X_preprocessed, train_y)

`braindecode`にはほかにも脳波デコードモデルが実装されている. 興味があれば下記公式ページを参照されたい.

- https://braindecode.org/stable/models/models_table.html

## 応募用ファイル作成

学習済みモデルで評価用データのラベルを予測し, 指定されたフォーマットで予測結果（提出用ファイル）をCSV形式で保存する.

In [ ]:
# 学習済みモデル（net）をCPUに移動
# （GPUで学習した場合でも、推論時にCPUで動かすことで予測が可能になる）
net.module_.cpu()

# 評価用データ（test_X_preprocessed）でラベル予測を行う
y_pred = net.predict(test_X_preprocessed)

# 予測ラベルとtest_idをまとめて提出用データフレームを作成
submit = pd.DataFrame({'test_id': test_id, 'y_pred': y_pred})

# データフレームをCSVファイルとして保存
# header=False, index=Falseとすることで余計な列名やインデックスを省略した提出用ファイルになる
submit.to_csv(os.path.join(OUTPUT_DIR, 'submit.csv'), header=False, index=False)

作成した`./submit.csv`をコンペティションサイトに投稿することで, スコアが計算され, リーダーボードに順位がついて確認ができる.

応募用ファイルの詳細なフォーマットについては`README.pdf`を参照すること.

## 深層学習の実装時によくある注意点・Tips


- **データのshape（形）に要注意！**  
    モデルが要求するshape（例：(サンプル数, チャンネル数, 系列長)）になっているかを, `.shape`で確認.

- **標準化などの前処理は、学習用・評価用どちらにも必ず同じ手順を！**  
    前処理の違いで急激に精度が落ちることがある.

- **モデルをGPUで学習した場合は、推論や保存時に `net.module_.cpu()` でCPUに戻すのを忘れずに！**

- **`batch_size`や`max_epochs`を大きくしすぎるとメモリ不足や学習が極端に遅くなることがあるので注意**

- **NaNやinfが出た場合は、データに異常値やゼロ割りがないか、標準化ロジックに問題がないかを確認する**

- **バリデーション（検証）データの割合は、おおよそ0.2～0.3が一般的（今回の0.25は標準的）**

- **深層学習モデルの学習は何度も試行錯誤！パラメータやネット構造をこまめに変えて実験しよう**


困ったときは「shapeの確認」「前処理の共通化」「エラーメッセージをよく読む」の3点をまず確認すること.

## 発展：さらに高度な手法にチャレンジしたい方へ

- **独自のニューラルネットワーク構造を自作してみたい**
    - 色々あるが, PyTorchが一般的によく使われるライブラリである.
    - PyTorchの`torch.nn.Module`を継承すれば, 自由なネットワーク（CNN, RNN, Attention付きなど）を0から定義できる.
    - 活性化関数や正則化（Dropoutなど）, 損失関数, 最適化アルゴリズムも自分で選んで組み合わせられる.

- **skorchやbraindecodeのラッパークラスではなく, PyTorch単体でモデル作成・学習ループも実装できる**
    - より細かく学習の流れをコントロールしたいときに有効.

- **モデルアーキテクチャの参考や実装はPyTorch公式チュートリアルやKaggleの上位ノートなどもおすすめ**
    - 公式：https://pytorch.org/tutorials/
    - Kaggle: https://www.kaggle.com/search?q=pytorch+notebook